In [1]:
from inverse_kinect import *
from common.h36m_dataset import Human36mDataset
from common.camera import *

import os,  inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
os.sys.path.insert(0,parentdir)

from pybullet_utils.bullet_client import BulletClient
from pybullet_envs.deep_mimic.motion_capture_data import MotionCaptureData
# import pybullet_data
import pybullet
import time
import random

from humanoid import Humanoid
from humanoid import HumanoidPose

import pybullet as p
import numpy as np

In [2]:
joint_info = {
	'joint_name':['root', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle', 'chest', 'neck', 'nose', 'eye', 'left_shoulder', 'left_elbow', 'left_wrist', 'right_shoulder', 'right_elbow', 'right_wrist'],
    'father':[0, 0, 1, 2, 0, 4, 5, 0, 7, 8, 9, 8, 11, 12, 8, 14, 15],
    'side':['middle', 'right', 'right', 'right', 'left', 'left', 'left', 'middle', 'middle', 'middle', 'middle', 'left', 'left', 'left', 'right', 'right', 'right'] 
}

In [3]:
def init_fb_h36m_dataset(dataset_path):
    dataset = Human36mDataset(dataset_path)
    print('Preparing Facebook H36M data...')
    for subject in dataset.subjects():
        for action in dataset[subject].keys():
            anim = dataset[subject][action]
            positions_3d = []
            for cam in anim['cameras']:
                pos_3d = world_to_camera(anim['positions'], R=cam['orientation'], t=cam['translation'])
                pos_3d[:, 1:] -= pos_3d[:, :1] # Remove global offset, but keep trajectory in first position
                positions_3d.append(pos_3d)
            anim['positions_3d'] = positions_3d
    return dataset

In [4]:
def init_fb_prediction_dataset(dataset_path):
    dataset_file = open(dataset_path,"rb")
    dataset = pickle.load(dataset_file)
    return dataset

In [5]:
def init_fb_prediction_json_dataset(dataset_path):
    dataset_file = open(dataset_path,"r")
    dataset = json.load(dataset_file)
    return dataset

In [6]:
def pose3D_from_fb_h36m(dataset, subject, action, shift):
    pose_seq = dataset[subject][action]['positions_3d'][0].copy()
    trajectory = pose_seq[:, :1]
    pose_seq[:, 1:] += trajectory
    # Invert camera transformation
    cam = dataset.cameras()[subject][0]
    pose_seq = camera_to_world(pose_seq, 
                                   R=cam['orientation'], 
                                   t=cam['translation'])
    x = pose_seq[:,:,0:1]
    y = pose_seq[:,:,1:2] * -1
    z = pose_seq[:,:,2:3] 
    pose_seq = np.concatenate((x,z,y),axis=2)
    # plus shift
    pose_seq += np.array([[shift for i in range(pose_seq.shape[1])] for j in range(pose_seq.shape[0])])
    return pose_seq

In [7]:
def pose3D_from_fb_prediction(dataset, subject, shift):
    should_right = [1,2,3,14,15,16]
    should_left = [4,5,6,11,12,13]
    pose_seq = dataset[subject]
    for i in range(pose_seq.shape[0]):
        for j in range(pose_seq.shape[1]):
            if i in should_right:
                t = pose_seq[i][j]
                change = should_left[should_right.index(i)]
                pose_seq[i][j] = pose_seq[change][j]
                pose_seq[change][j] = t
    for i in range(pose_seq.shape[0]):
        for j in range(pose_seq.shape[1]):
            pose_seq[i][j][1] *= -1
            pose_seq[i][j][2] *= -1

    pose_seq += np.array([[shift for i in range(pose_seq.shape[1])] for j in range(pose_seq.shape[0])])
    return pose_seq

In [8]:
def pose3D_from_fb_prediction_json(dataset, shift):
    should_right = [1,2,3,14,15,16]
    should_left = [4,5,6,11,12,13]
    pose_seq = np.array(dataset['pose3d'])
    for i in range(pose_seq.shape[0]):
        for j in range(pose_seq.shape[1]):
            if i in should_right:
                t = pose_seq[i][j]
                change = should_left[should_right.index(i)]
                pose_seq[i][j] = pose_seq[change][j]
                pose_seq[change][j] = t
    for i in range(pose_seq.shape[0]):
        for j in range(pose_seq.shape[1]):
            pose_seq[i][j][1] *= -1
            pose_seq[i][j][2] *= -1

    pose_seq += np.array([[shift for i in range(pose_seq.shape[1])] for j in range(pose_seq.shape[0])])
    return pose_seq

In [9]:
def rot_seq_to_deepmimic_json(rot_seq, loop, json_path):
    to_json = {"Loop": loop, "Frames":[]}
    rot_seq = np.around(rot_seq, decimals=6)
    to_json["Frames"] = rot_seq.tolist()
    # In[14]:
    to_file = json.dumps(to_json)
    file = open(json_path,"w")
    file.write(to_file)
    file.close()

In [10]:
def draw_ground_truth(coord_seq, frame, duration, shift):
    global joint_info
    joint = coord_seq[frame]
    shift = np.array(shift)
    for i in range(1, 17):
        # print(x[11], x[14])
        joint_fa = joint_info['father'][i]
        if joint_info['side'][i] == 'right':
            p.addUserDebugLine(lineFromXYZ=joint[i]+shift,
                               lineToXYZ=joint[joint_fa]+shift,
                               lineColorRGB=(255,0,0),
                               lineWidth=1,
                               lifeTime=duration)
        else:
            p.addUserDebugLine(lineFromXYZ=joint[i]+shift,
                               lineToXYZ=joint[joint_fa]+shift,
                               lineColorRGB=(0,0,0),
                               lineWidth=1,
                               lifeTime=duration)

In [11]:
fb_h36m_dataset_path = 'data/data_3d_h36m.npz'
dataset = init_fb_h36m_dataset(fb_h36m_dataset_path)
ground_truth = pose3D_from_fb_h36m(dataset, 
                                   subject = 'S11', 
                                   action = 'Walking',
                                   shift = [1.0,0.0,0.0])

# # if load data from facebook output, set y-shift to 0.85
# fb_prediction_dataset_path = 'data/tmp_3d_pose_data.pkl'
# dataset = init_fb_prediction_dataset(fb_prediction_dataset_path)
# ground_truth = pose3D_from_fb_prediction(  dataset, 
#                                            subject = 1, 
#                                            shift = [1.0,0.85,0.0])

# # if load data from facebook output, set y-shift to 0.85
# fb_prediction_json_dataset_path = 'data/pose3d-results-0.json'
# dataset = init_fb_prediction_json_dataset(fb_prediction_json_dataset_path)
# ground_truth = pose3D_from_fb_prediction_json(  dataset, 
#                                                 shift = [1.0,0.85,0.0])

coord_seq = ground_truth
rot_seq =  coord_seq_to_rot_seq(coord_seq = coord_seq, 
                                frame_duration = 1/24)

Preparing Facebook H36M data...


In [12]:
# t is threshold
def rotation_sequence_clean(coord_seq, rot_seq, threshold):
    clean_coord_seq = []
    clean_rot_seq = []
    for i in range(1, len(rot_seq)):
        now = np.array(rot_seq[i])        
        last = np.array(rot_seq[i-1])
        dist = np.linalg.norm(now-last)
#         test_mul = np.dot(now, last) / now.shape[0]
        print(i, dist)
        if dist <= threshold:
            clean_rot_seq.append(rot_seq[i])
            clean_coord_seq.append(coord_seq[i])
    return [clean_coord_seq, clean_rot_seq]

In [13]:
coord_seq, rot_seq = rotation_sequence_clean(coord_seq, rot_seq, threshold = 0.4)

1 0.0011100759457470138
2 0.0016133241013324548
3 0.003217655857004723
4 0.00742686560344155
5 0.005618930619129623
6 0.005557842534927249
7 0.0075979792084581
8 0.005912395855601615
9 0.007693042197153241
10 0.011176941496809673
11 0.01419162818881518
12 0.019250909234036745
13 0.024786322904311677
14 0.031111640380918588
15 0.04131137160668588
16 0.04556445200570449
17 0.051224575120853356
18 0.06170610927566375
19 0.05909067537841891
20 0.06592146202380153
21 0.06479168840169529
22 0.06470755419115673
23 0.059002263485011225
24 0.06118246908163844
25 0.06391437835663986
26 0.0680288947573528
27 0.07720251068236336
28 0.07948596176817137
29 0.07904648896985515
30 0.07446216032822638
31 0.07443244283970843
32 0.07170837959308922
33 0.07206239209075357
34 0.07380897990106605
35 0.0732002525075686
36 0.06417233195605757
37 0.05872777151078543
38 0.05586226922115153
39 0.0565966849597022
40 0.05863970052205649
41 0.05995231077835197
42 0.057117768548306344
43 0.05840724441210975
44 0.056

960 0.06726084747102365
961 0.05521704865069144
962 0.0456997649541272
963 0.04529870533696168
964 0.049918093293352166
965 0.06051597855123716
966 0.0720420649685907
967 0.08191215701703154
968 0.09084446165446028
969 0.09695088234950094
970 0.10697809914229582
971 0.11496015127347055
972 0.12288895552952216
973 0.12543804114157994
974 0.11916387493131678
975 0.10300033003390967
976 0.06689529682765373
977 0.03856597693598974
978 0.05434181085745491
979 0.07195939335773398
980 0.06462541941295899
981 0.07123086886118053
982 0.0898475708265878
983 0.11233107393457928
984 0.12273631036987677
985 0.12177119566289127
986 0.11585192668755069
987 0.1032061041539374
988 0.09121812617600447
989 0.07202856973129293
990 0.05920155436671656
991 0.04527052798732623
992 0.03877988441923257
993 0.038395357267430465
994 0.04386573625894242
995 0.052174362184240215
996 0.05757966545118601
997 0.06622278677908426
998 0.07505649686144357
999 0.08459699982132513
1000 0.09478826029835244
1001 0.103823938

In [14]:
rot_seq_to_deepmimic_json(  rot_seq = rot_seq, 
                            loop = 'wrap',
                            json_path = 'data/test_dance.json')

In [15]:
bc = BulletClient(connection_mode=pybullet.GUI)
# bc.setAdditionalSearchPath(os.getcwd())
bc.configureDebugVisualizer(bc.COV_ENABLE_Y_AXIS_UP,1)
bc.setGravity(0,-9.8,0)
motion=MotionCaptureData()

# motionPath = pybullet_data.getDataPath()+"/motions/humanoid3d_backflip.txt"#humanoid3d_spinkick.txt"#/motions/humanoid3d_backflip.txt"
motionPath = 'data/test_dance.json'
motion.Load(motionPath)
print("numFrames = ", motion.NumFrames())

numFrames =  1616


In [16]:
simTimeId= bc.addUserDebugParameter("simTime",0,motion.NumFrames()-1.1,0)

y2zOrn = bc.getQuaternionFromEuler([-1.57,0,0])
bc.loadURDF("data/plane.urdf",[0,-0.04,0], y2zOrn)

# humanoid = Humanoid(bc, motion, [0,0,0])#4000,0,5000])
humanoid = Humanoid(bc, motion, [0,0,0]) #这是初始位置的坐标

LOADING humanoid!


In [17]:
print(p.getBasePositionAndOrientation(humanoid._humanoid))

((0.964478, 0.982553, 0.201419), (-0.0304879848766138, -0.7451616116538676, 0.0001629999170714549, 0.6661866320533615))


In [18]:
simTime = 0
keyFrameDuration = motion.KeyFrameDuraction()
print("keyFrameDuration=",keyFrameDuration)
for utNum in range(motion.NumFrames()):
    bc.stepSimulation()
    # humanoid.ApplyPose(pose, True, True, humanoid._humanoid,bc)
    humanoid.RenderReference(utNum * keyFrameDuration)
    draw_ground_truth(coord_seq = coord_seq, 
                      frame = utNum, 
                      duration = keyFrameDuration,
                      shift = [-1.0, 0.0, 1.0])
    time.sleep(0.001)
#     print(utNum, motion._motion_data['Frames'][utNum][4:8])
stage = 0

keyFrameDuration= 0.041667


error: Not connected to physics server.

In [ ]:
def Reset(humanoid):
	global simTime
	humanoid.Reset()
	simTime = 0 #random.randint(0,motion.NumFrames()-2)
	humanoid.SetSimTime(simTime)
	pose = humanoid.InitializePoseFromMotionData()
	humanoid.ApplyPose(pose, True, True, humanoid._humanoid,bc)


In [ ]:
Reset(humanoid)
p.disconnect()
#bc.stepSimulation()